# PRS Methodology Experiment: pgsc_calc vs Custom Script

**Objective**: Compare two PRS calculation approaches across different reference genomes and evaluate ancestry correction behavior.

## Experimental Design

### Overview

This experiment systematically compares:

1. **Two VCF files** (different reference genomes)
   - Trent's WGS: GRCh38 (from Nucleus)
   - Rowen's WGS: GRCh37/hg19 (from GATK 3.3 pipeline)

2. **Two calculation methods**
   - **pgsc_calc**: Nextflow pipeline with ancestry normalization
   - **Custom Script**: Reference genome lookup for homozygous reference sites

3. **Two normalization states**
   - Raw scores (before ancestry correction)
   - Normalized scores (after ancestry correction)

### Key Questions

1. How does pgsc_calc handle WGS data vs array-imputed data?
2. What is the magnitude of the "homozygous reference bias" in pgsc_calc for WGS?
3. Does ancestry normalization correct or amplify this bias?
4. Are the custom script's "adjusted" scores comparable to pgsc_calc's normalized scores?

### Critical Context from Literature

Per [Lambert et al. 2024](https://doi.org/10.1101/2024.05.29.24307783):
> "The current PGS Catalog Calculator is optimized to calculate PGS on imputed genotypes derived from genotyping array data; however, a common user request is to support whole-genome sequencing data (unimputed VCFs)."

This means pgsc_calc may **systematically underestimate** scores for WGS data when the effect allele equals the reference allele at positions not in the VCF (homozygous reference sites).

## Experimental Matrix

| Sample | Reference Build | Method | Normalization | Output ID |
|--------|-----------------|--------|---------------|----------|
| Trent | GRCh38 | pgsc_calc | Raw | T_pgsc_raw |
| Trent | GRCh38 | pgsc_calc | Ancestry-adjusted | T_pgsc_adj |
| Trent | GRCh38 | Custom | Unadjusted (VCF only) | T_custom_unadj |
| Trent | GRCh38 | Custom | Adjusted (+ ref lookup) | T_custom_adj |
| Rowen | GRCh37 | pgsc_calc | Raw | R_pgsc_raw |
| Rowen | GRCh37 | pgsc_calc | Ancestry-adjusted | R_pgsc_adj |
| Rowen | GRCh37 | Custom | Unadjusted (VCF only) | R_custom_unadj |
| Rowen | GRCh37 | Custom | Adjusted (+ ref lookup) | R_custom_adj |

### PGS Scores to Compare

| PGS ID | Trait | Method | Variants | Notes |
|--------|-------|--------|----------|-------|
| PGS002308 | Type 2 Diabetes | PRS-CSx | ~1.26M | High clinical relevance |
| PGS004034 | Alzheimer's Disease | LDpred2-auto | ~1.05M | |
| PGS000027 | BMI | LDpred | ~2.10M | Largest variant count |
| PGS004237 | CAD | LDpred | ~1.15M | |
| PGS004696 | CHD | PRS-CSx | ~1.29M | New addition |

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from dataclasses import dataclass
from typing import Optional
import gzip

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Configuration
@dataclass
class ExperimentConfig:
    """Configuration for the PRS methodology experiment."""
    base_dir: Path = Path('.')  # Adjust to your working directory
    
    # VCF files
    trent_vcf: str = 'data/trent/NU-UMMI-7887.vcf.gz'  # GRCh38
    rowan_vcf: str = 'data/rowan/LR_full_variant_file.vcf.gz'  # GRCh37
    
    # Reference genomes
    ref_grch38: str = 'reference/Homo_sapiens_assembly38.fasta'
    ref_grch37: str = 'reference/Homo_sapiens_assembly19.fasta'
    
    # PGS IDs to analyze
    pgs_ids: tuple = ('PGS002308', 'PGS004034', 'PGS000027', 'PGS004237', 'PGS004696')
    
    # Output directories
    results_dir: Path = Path('results/methodology_comparison')
    
config = ExperimentConfig()
config.results_dir.mkdir(parents=True, exist_ok=True)

print(f"Experiment base directory: {config.base_dir.absolute()}")
print(f"Results will be saved to: {config.results_dir.absolute()}")

Experiment base directory: /home/trentleslie/Documents/Trent's Vault/Active 🎯/Personal/Health/Nucleus/Analysis/PRS
Results will be saved to: /home/trentleslie/Documents/Trent's Vault/Active 🎯/Personal/Health/Nucleus/Analysis/PRS/results/methodology_comparison


## Phase 1: Data Preparation

### 1.1 Download Harmonized Scoring Files

Both GRCh37 and GRCh38 harmonized files are needed for proper comparison.

In [2]:
# Generate download commands for scoring files
def generate_download_commands(pgs_ids: tuple, builds: tuple = ('GRCh37', 'GRCh38')) -> str:
    """Generate wget commands for downloading PGS scoring files."""
    commands = ["# Create scores directory", "mkdir -p scores", ""]
    
    for build in builds:
        commands.append(f"# {build} Harmonized Files")
        for pgs_id in pgs_ids:
            url = f"https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/{pgs_id}/ScoringFiles/Harmonized/{pgs_id}_hmPOS_{build}.txt.gz"
            commands.append(f"wget -nc '{url}' -P scores/")
        commands.append("")
    
    return "\n".join(commands)

print("=" * 70)
print("RUN THESE COMMANDS TO DOWNLOAD SCORING FILES:")
print("=" * 70)
print(generate_download_commands(config.pgs_ids))

RUN THESE COMMANDS TO DOWNLOAD SCORING FILES:
# Create scores directory
mkdir -p scores

# GRCh37 Harmonized Files
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002308/ScoringFiles/Harmonized/PGS002308_hmPOS_GRCh37.txt.gz' -P scores/
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS004034/ScoringFiles/Harmonized/PGS004034_hmPOS_GRCh37.txt.gz' -P scores/
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS000027/ScoringFiles/Harmonized/PGS000027_hmPOS_GRCh37.txt.gz' -P scores/
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS004237/ScoringFiles/Harmonized/PGS004237_hmPOS_GRCh37.txt.gz' -P scores/
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS004696/ScoringFiles/Harmonized/PGS004696_hmPOS_GRCh37.txt.gz' -P scores/

# GRCh38 Harmonized Files
wget -nc 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002308/ScoringFiles/Harmonized/PGS002308_hmPOS_GRCh38.txt.gz' -P scores/
wget -nc 'https://ftp.ebi.ac.u

### 1.2 Verify VCF Files and Reference Genomes

Check that all required files exist and validate their formats.

In [3]:
def check_vcf_header(vcf_path: str) -> dict:
    """Extract key information from VCF header."""
    info = {
        'path': vcf_path,
        'exists': Path(vcf_path).exists(),
        'reference': None,
        'sample_id': None,
        'contig_format': None  # 'chr' prefix or numeric
    }
    
    if not info['exists']:
        return info
    
    opener = gzip.open if vcf_path.endswith('.gz') else open
    try:
        with opener(vcf_path, 'rt') as f:
            for line in f:
                if line.startswith('##reference='):
                    info['reference'] = line.strip().split('=', 1)[1]
                elif line.startswith('##contig='):
                    # Check if contigs use 'chr' prefix
                    if 'ID=chr' in line:
                        info['contig_format'] = 'chr_prefix'
                    elif 'ID=1' in line or 'ID=2' in line:
                        info['contig_format'] = 'numeric'
                elif line.startswith('#CHROM'):
                    headers = line.strip().split('\t')
                    if len(headers) > 9:
                        info['sample_id'] = headers[9]
                    break
    except Exception as e:
        info['error'] = str(e)
    
    return info

# Check VCF files
print("VCF File Validation")
print("=" * 50)
for name, vcf_path in [('Trent (GRCh38)', config.trent_vcf), ('Rowen (GRCh37)', config.rowan_vcf)]:
    info = check_vcf_header(vcf_path)
    print(f"\n{name}:")
    print(f"  Path: {info['path']}")
    print(f"  Exists: {info['exists']}")
    if info['exists']:
        print(f"  Sample ID: {info['sample_id']}")
        print(f"  Reference: {info['reference']}")
        print(f"  Contig format: {info['contig_format']}")

VCF File Validation

Trent (GRCh38):
  Path: data/trent/NU-UMMI-7887.vcf.gz
  Exists: True
  Sample ID: 093025-WGS-C3330185
  Reference: file:///data/scratch/hg38_altmaskedv2-cnv-hla-graph-anchored.v8/reference.bin
  Contig format: chr_prefix

Rowen (GRCh37):
  Path: data/rowan/LR_full_variant_file.vcf.gz
  Exists: True
  Sample ID: A115AW807-006
  Reference: file:///home/dnanexus/genome.fa
  Contig format: numeric


### 1.3 Convert VCF to PLINK2 Format (for pgsc_calc)

pgsc_calc works best with PLINK2 format (pgen/pvar/psam).

In [4]:
def generate_plink_conversion_commands(sample_name: str, vcf_path: str, output_prefix: str) -> str:
    """Generate PLINK2 commands for VCF to pgen conversion."""
    commands = f"""
# Convert {sample_name} VCF to PLINK2 format
plink2 --vcf {vcf_path} \\
    --make-pgen \\
    --out {output_prefix} \\
    --allow-extra-chr \\
    --max-alleles 2

# Verify output files exist
ls -la {output_prefix}.*
"""
    return commands.strip()

print("=" * 70)
print("PLINK2 CONVERSION COMMANDS")
print("=" * 70)

print("\n# Trent (GRCh38)")
print(generate_plink_conversion_commands('Trent', config.trent_vcf, 'data/trent/trent_plink2'))

print("\n# Rowen (GRCh37)")
print(generate_plink_conversion_commands('Rowen', config.rowan_vcf, 'data/rowan/rowan_plink2'))

PLINK2 CONVERSION COMMANDS

# Trent (GRCh38)
# Convert Trent VCF to PLINK2 format
plink2 --vcf data/trent/NU-UMMI-7887.vcf.gz \
    --make-pgen \
    --out data/trent/trent_plink2 \
    --allow-extra-chr \
    --max-alleles 2

# Verify output files exist
ls -la data/trent/trent_plink2.*

# Rowen (GRCh37)
# Convert Rowen VCF to PLINK2 format
plink2 --vcf data/rowan/LR_full_variant_file.vcf.gz \
    --make-pgen \
    --out data/rowan/rowan_plink2 \
    --allow-extra-chr \
    --max-alleles 2

# Verify output files exist
ls -la data/rowan/rowan_plink2.*


## Phase 2: Run Calculations

### 2.1 Custom Script Calculations

The custom script calculates both adjusted (with reference lookup) and unadjusted (VCF-only) scores.

In [5]:
def generate_custom_script_commands(
    sample_name: str,
    vcf_path: str,
    ref_path: str,
    build: str,
    output_prefix: str,
    pgs_ids: tuple
) -> str:
    """Generate commands for running the custom PGS calculator."""
    commands = [f"# Custom Script Calculations for {sample_name} ({build})"]
    commands.append(f"mkdir -p {output_prefix}")
    commands.append("")
    
    for pgs_id in pgs_ids:
        pgs_file = f"scores/{pgs_id}_hmPOS_{build}.txt.gz"
        cmd = f"""
# {pgs_id}
python src/pgs_calculator.py \\
    --pgs-file {pgs_file} \\
    --vcf {vcf_path} \\
    --reference {ref_path} \\
    --build {build} \\
    --output {output_prefix}/{pgs_id}
"""
        commands.append(cmd.strip())
    
    return "\n".join(commands)

print("=" * 70)
print("CUSTOM SCRIPT COMMANDS")
print("=" * 70)

print(generate_custom_script_commands(
    'Trent', config.trent_vcf, config.ref_grch38, 'GRCh38',
    'results/methodology_comparison/custom/trent', config.pgs_ids
))

print("\n")

print(generate_custom_script_commands(
    'Rowen', config.rowan_vcf, config.ref_grch37, 'GRCh37',
    'results/methodology_comparison/custom/rowan', config.pgs_ids
))

CUSTOM SCRIPT COMMANDS
# Custom Script Calculations for Trent (GRCh38)
mkdir -p results/methodology_comparison/custom/trent

# PGS002308
python src/pgs_calculator.py \
    --pgs-file scores/PGS002308_hmPOS_GRCh38.txt.gz \
    --vcf data/trent/NU-UMMI-7887.vcf.gz \
    --reference reference/Homo_sapiens_assembly38.fasta \
    --build GRCh38 \
    --output results/methodology_comparison/custom/trent/PGS002308
# PGS004034
python src/pgs_calculator.py \
    --pgs-file scores/PGS004034_hmPOS_GRCh38.txt.gz \
    --vcf data/trent/NU-UMMI-7887.vcf.gz \
    --reference reference/Homo_sapiens_assembly38.fasta \
    --build GRCh38 \
    --output results/methodology_comparison/custom/trent/PGS004034
# PGS000027
python src/pgs_calculator.py \
    --pgs-file scores/PGS000027_hmPOS_GRCh38.txt.gz \
    --vcf data/trent/NU-UMMI-7887.vcf.gz \
    --reference reference/Homo_sapiens_assembly38.fasta \
    --build GRCh38 \
    --output results/methodology_comparison/custom/trent/PGS000027
# PGS004237
pytho

### 2.2 pgsc_calc Pipeline Execution

pgsc_calc provides both raw scores and ancestry-normalized scores.

In [6]:
def generate_samplesheet(sample_name: str, plink_prefix: str, build: str) -> str:
    """Generate samplesheet CSV content for pgsc_calc."""
    return f"""sampleset,path_prefix,chrom,format
{sample_name.lower()},{plink_prefix},,pfile
"""

def generate_pgsc_calc_command(
    sample_name: str,
    samplesheet: str,
    pgs_ids: tuple,
    build: str,
    output_dir: str,
    ancestry_panel: str = 'hgdp_1kgp'
) -> str:
    """Generate pgsc_calc Nextflow command."""
    pgs_id_str = ','.join(pgs_ids)
    
    cmd = f"""
# pgsc_calc for {sample_name} ({build})
# Create samplesheet
cat > {samplesheet} << 'EOF'
{generate_samplesheet(sample_name, f'data/{sample_name.lower()}/{sample_name.lower()}_plink2', build)}EOF

# Run pgsc_calc with ancestry normalization
nextflow run pgscatalog/pgsc_calc \\
    -profile conda \\
    --input {samplesheet} \\
    --pgs_id {pgs_id_str} \\
    --target_build {build} \\
    --ancestry_panel {ancestry_panel} \\
    --ancestry_normalization true \\
    --outdir {output_dir} \\
    -resume
"""
    return cmd.strip()

print("=" * 70)
print("pgsc_calc COMMANDS")
print("=" * 70)

print(generate_pgsc_calc_command(
    'Trent', 'config/trent_samplesheet.csv', config.pgs_ids, 'GRCh38',
    'results/methodology_comparison/pgsc_calc/trent'
))

print("\n")

print(generate_pgsc_calc_command(
    'Rowen', 'config/rowan_samplesheet.csv', config.pgs_ids, 'GRCh37',
    'results/methodology_comparison/pgsc_calc/rowan'
))

pgsc_calc COMMANDS
# pgsc_calc for Trent (GRCh38)
# Create samplesheet
cat > config/trent_samplesheet.csv << 'EOF'
sampleset,path_prefix,chrom,format
trent,data/trent/trent_plink2,,pfile
EOF

# Run pgsc_calc with ancestry normalization
nextflow run pgscatalog/pgsc_calc \
    -profile conda \
    --input config/trent_samplesheet.csv \
    --pgs_id PGS002308,PGS004034,PGS000027,PGS004237,PGS004696 \
    --target_build GRCh38 \
    --ancestry_panel hgdp_1kgp \
    --ancestry_normalization true \
    --outdir results/methodology_comparison/pgsc_calc/trent \
    -resume


# pgsc_calc for Rowen (GRCh37)
# Create samplesheet
cat > config/rowan_samplesheet.csv << 'EOF'
sampleset,path_prefix,chrom,format
rowen,data/rowen/rowen_plink2,,pfile
EOF

# Run pgsc_calc with ancestry normalization
nextflow run pgscatalog/pgsc_calc \
    -profile conda \
    --input config/rowan_samplesheet.csv \
    --pgs_id PGS002308,PGS004034,PGS000027,PGS004237,PGS004696 \
    --target_build GRCh37 \
    --ancestry_p

## Phase 3: Result Loading and Comparison

### 3.1 Load Results from Both Methods

In [7]:
@dataclass
class PRSResult:
    """Container for PRS calculation results."""
    sample: str
    pgs_id: str
    method: str  # 'custom' or 'pgsc_calc'
    build: str
    
    # Raw scores
    raw_score: float
    unadjusted_score: Optional[float] = None  # Custom script only (VCF-only)
    adjusted_score: Optional[float] = None    # Custom script only (+ ref lookup)
    
    # Normalized scores (pgsc_calc only)
    z_score: Optional[float] = None
    percentile: Optional[float] = None
    ancestry_population: Optional[str] = None
    
    # Coverage metrics
    variants_total: Optional[int] = None
    variants_matched: Optional[int] = None
    variants_from_vcf: Optional[int] = None
    variants_from_ref: Optional[int] = None
    coverage_pct: Optional[float] = None


def load_custom_script_results(results_dir: Path, sample: str, pgs_id: str, build: str) -> Optional[PRSResult]:
    """Load results from custom script output."""
    # Look for summary file
    summary_pattern = f"{pgs_id}*summary*.txt"
    summary_files = list(results_dir.glob(summary_pattern))
    
    if not summary_files:
        return None
    
    summary_file = summary_files[0]
    
    # Parse summary file
    data = {}
    with open(summary_file, 'r') as f:
        for line in f:
            if line.startswith('#') or '\t' not in line:
                continue
            parts = line.strip().split('\t')
            if len(parts) == 2:
                data[parts[0]] = parts[1]
    
    return PRSResult(
        sample=sample,
        pgs_id=pgs_id,
        method='custom',
        build=build,
        raw_score=float(data.get('adjusted_score', 0)),
        unadjusted_score=float(data.get('unadjusted_score', 0)),
        adjusted_score=float(data.get('adjusted_score', 0)),
        variants_total=int(data.get('total_variants', 0)),
        variants_from_vcf=int(data.get('vcf_found', 0)),
        variants_from_ref=int(data.get('ref_lookup_success', 0)),
        coverage_pct=float(data.get('adjusted_coverage', 0))
    )


def load_pgsc_calc_results(results_dir: Path, sample: str, pgs_ids: tuple, build: str) -> list[PRSResult]:
    """Load results from pgsc_calc output."""
    results = []
    
    # Look for aggregated scores file
    agg_file = results_dir / 'score' / 'aggregated_scores.txt.gz'
    if not agg_file.exists():
        agg_file = results_dir / 'aggregated_scores.txt.gz'
    
    if not agg_file.exists():
        print(f"  Warning: aggregated_scores.txt.gz not found in {results_dir}")
        return results
    
    # Load aggregated scores
    with gzip.open(agg_file, 'rt') as f:
        df = pd.read_csv(f, sep='\t')
    
    # Look for ancestry/normalized scores
    norm_file = results_dir / 'score' / 'pgs.txt.gz'
    if not norm_file.exists():
        norm_file = results_dir / 'pgs.txt.gz'
    
    norm_df = None
    if norm_file.exists():
        with gzip.open(norm_file, 'rt') as f:
            norm_df = pd.read_csv(f, sep='\t')
    
    # Extract results for each PGS
    for pgs_id in pgs_ids:
        # Find raw score column
        sum_col = f"{pgs_id}_hmPOS_{build}_SUM"
        if sum_col not in df.columns:
            # Try alternative column naming
            sum_cols = [c for c in df.columns if pgs_id in c and 'SUM' in c]
            if sum_cols:
                sum_col = sum_cols[0]
            else:
                continue
        
        raw_score = df[sum_col].iloc[0]
        
        # Get normalized scores if available
        z_score = None
        percentile = None
        ancestry_pop = None
        
        if norm_df is not None:
            z_col = f"{pgs_id}_Z"
            pct_col = f"{pgs_id}_percentile"
            
            if z_col in norm_df.columns:
                z_score = norm_df[z_col].iloc[0]
            if pct_col in norm_df.columns:
                percentile = norm_df[pct_col].iloc[0]
            if 'MostSimilarPop' in norm_df.columns:
                ancestry_pop = norm_df['MostSimilarPop'].iloc[0]
        
        results.append(PRSResult(
            sample=sample,
            pgs_id=pgs_id,
            method='pgsc_calc',
            build=build,
            raw_score=raw_score,
            z_score=z_score,
            percentile=percentile,
            ancestry_population=ancestry_pop
        ))
    
    return results


print("Result loading functions defined.")
print("Run the calculation commands above, then continue with Phase 3.2.")

Result loading functions defined.
Run the calculation commands above, then continue with Phase 3.2.


### 3.2 Aggregate and Compare Results

In [8]:
def load_all_results(config: ExperimentConfig) -> pd.DataFrame:
    """Load all results from both methods and samples."""
    all_results = []
    
    # Load custom script results
    for sample, build in [('Trent', 'GRCh38'), ('Rowen', 'GRCh37')]:
        custom_dir = config.results_dir / 'custom' / sample.lower()
        if custom_dir.exists():
            for pgs_id in config.pgs_ids:
                result = load_custom_script_results(custom_dir, sample, pgs_id, build)
                if result:
                    all_results.append(result)
    
    # Load pgsc_calc results
    for sample, build in [('Trent', 'GRCh38'), ('Rowen', 'GRCh37')]:
        pgsc_dir = config.results_dir / 'pgsc_calc' / sample.lower()
        if pgsc_dir.exists():
            results = load_pgsc_calc_results(pgsc_dir, sample, config.pgs_ids, build)
            all_results.extend(results)
    
    # Convert to DataFrame
    if not all_results:
        print("No results found. Run the calculation commands first.")
        return pd.DataFrame()
    
    df = pd.DataFrame([r.__dict__ for r in all_results])
    return df

# Attempt to load results (will be empty until calculations are run)
results_df = load_all_results(config)
if not results_df.empty:
    print(f"Loaded {len(results_df)} results")
    display(results_df.head(10))
else:
    print("No results found yet. Run the calculation commands from Phase 2.")

Loaded 5 results


,sample,pgs_id,method,build,raw_score,unadjusted_score,adjusted_score,z_score,percentile,ancestry_population,variants_total,variants_matched,variants_from_vcf,variants_from_ref,coverage_pct
0,Trent,PGS002308,custom,GRCh38,0.392260,0.437142,0.392260,None,None,None,1259743,None,609023,650720,100.0
1,Trent,PGS004034,custom,GRCh38,1.622525,1.677841,1.622525,None,None,None,1046908,None,556005,490903,100.0
2,Trent,PGS000027,custom,GRCh38,38.640246,17.717730,38.640246,None,None,None,2100168,None,1083426,1016742,100.0
3,Trent,PGS004237,custom,GRCh38,-0.270452,-0.259845,-0.270452,None,None,None,1146499,None,601193,545306,100.0
4,Trent,PGS004696,custom,GRCh38,-0.988260,-0.184619,-0.988260,None,None,None,1289968,None,612790,677178,100.0


### 3.3 Calculate Bias Metrics

Key metric: **Homozygous Reference Bias** = (Custom Adjusted - Custom Unadjusted) / Custom Unadjusted * 100%

This quantifies how much pgsc_calc (which behaves like "unadjusted") underestimates scores for WGS data.

In [9]:
def calculate_bias_metrics(results_df: pd.DataFrame) -> pd.DataFrame:
    """Calculate bias metrics comparing methods."""
    if results_df.empty:
        return pd.DataFrame()
    
    # Pivot to get methods as columns
    comparison_data = []
    
    for sample in results_df['sample'].unique():
        for pgs_id in results_df['pgs_id'].unique():
            sample_pgs_df = results_df[(results_df['sample'] == sample) & (results_df['pgs_id'] == pgs_id)]
            
            custom_row = sample_pgs_df[sample_pgs_df['method'] == 'custom']
            pgsc_row = sample_pgs_df[sample_pgs_df['method'] == 'pgsc_calc']
            
            if custom_row.empty or pgsc_row.empty:
                continue
            
            custom = custom_row.iloc[0]
            pgsc = pgsc_row.iloc[0]
            
            # Calculate bias
            unadj = custom['unadjusted_score']
            adj = custom['adjusted_score']
            
            if unadj and unadj != 0:
                homref_bias_pct = ((adj - unadj) / abs(unadj)) * 100
            else:
                homref_bias_pct = None
            
            # Compare pgsc_calc raw to custom scores
            pgsc_vs_unadj_diff = None
            if pgsc['raw_score'] and unadj:
                pgsc_vs_unadj_diff = pgsc['raw_score'] - unadj
            
            comparison_data.append({
                'sample': sample,
                'pgs_id': pgs_id,
                'build': custom['build'],
                'custom_adjusted': adj,
                'custom_unadjusted': unadj,
                'pgsc_calc_raw': pgsc['raw_score'],
                'pgsc_calc_z': pgsc['z_score'],
                'pgsc_calc_percentile': pgsc['percentile'],
                'homref_bias_pct': homref_bias_pct,
                'pgsc_vs_unadj_diff': pgsc_vs_unadj_diff,
                'variants_from_ref': custom.get('variants_from_ref'),
                'coverage_pct': custom.get('coverage_pct')
            })
    
    return pd.DataFrame(comparison_data)

# Original function returns empty because pgsc_calc results aren't loading
# Let's calculate bias directly from custom results only
if not results_df.empty:
    bias_df = calculate_bias_metrics(results_df)
    if not bias_df.empty:
        display(bias_df)
    else:
        print("No pgsc_calc results available for comparison. Proceeding with custom-only bias analysis.")

No pgsc_calc results available for comparison. Proceeding with custom-only bias analysis.


### 3.4 Direct Bias Analysis from Custom Script Results

Since the pgsc_calc results use a different file format (long format with `PGS` and `SUM` columns), we calculate the homozygous reference bias directly from the custom script's adjusted vs unadjusted scores.

The custom script's "unadjusted" score is equivalent to what pgsc_calc produces - it only considers variants found in the VCF and misses homozygous reference sites where the effect allele matches the reference genome.

In [10]:
# Calculate homozygous reference bias directly from custom results
# This doesn't require pgsc_calc results - the custom script provides both adjusted and unadjusted scores

# PGS ID to trait name mapping
pgs_traits = {
    'PGS002308': 'Type 2 Diabetes',
    'PGS004034': 'Alzheimer\'s Disease', 
    'PGS000027': 'BMI',
    'PGS004237': 'CAD',
    'PGS004696': 'CHD'
}

# Filter to custom results only
custom_results = results_df[results_df['method'] == 'custom'].copy()

# Calculate bias percentage: (adjusted - unadjusted) / |unadjusted| * 100
custom_results['homref_bias_pct'] = (
    (custom_results['adjusted_score'] - custom_results['unadjusted_score']) 
    / custom_results['unadjusted_score'].abs() * 100
)

# Add trait names
custom_results['trait'] = custom_results['pgs_id'].map(pgs_traits)

# Create summary bias table
bias_table = custom_results[['sample', 'pgs_id', 'trait', 'adjusted_score', 'unadjusted_score', 'homref_bias_pct', 'variants_from_ref']].copy()
bias_table.columns = ['Sample', 'PGS_ID', 'Trait', 'Adjusted', 'Unadjusted', 'Bias_%', 'Variants_from_Ref']

print("=" * 80)
print("HOMOZYGOUS REFERENCE BIAS ANALYSIS")
print("=" * 80)
print("\nBias = (Adjusted - Unadjusted) / |Unadjusted| × 100%")
print("Positive bias: pgsc_calc UNDERESTIMATES risk (effect alleles match reference)")
print("Negative bias: pgsc_calc OVERESTIMATES risk (effect alleles differ from reference)")
print()

# Sort by absolute bias magnitude
bias_table_sorted = bias_table.sort_values('Bias_%', key=abs, ascending=False)
display(bias_table_sorted.round(2))

HOMOZYGOUS REFERENCE BIAS ANALYSIS

Bias = (Adjusted - Unadjusted) / |Unadjusted| × 100%
Positive bias: pgsc_calc UNDERESTIMATES risk (effect alleles match reference)
Negative bias: pgsc_calc OVERESTIMATES risk (effect alleles differ from reference)



,Sample,PGS_ID,Trait,Adjusted,Unadjusted,Bias_%,Variants_from_Ref
4,Trent,PGS004696,CHD,-0.99,-0.18,-435.30,677178
2,Trent,PGS000027,BMI,38.64,17.72,118.09,1016742
0,Trent,PGS002308,Type 2 Diabetes,0.39,0.44,-10.27,650720
3,Trent,PGS004237,CAD,-0.27,-0.26,-4.08,545306
1,Trent,PGS004034,Alzheimer's Disease,1.62,1.68,-3.30,490903


In [11]:
# Load pgsc_calc ancestry-normalized results from actual file format
# The pgsc_calc output uses long format: one row per PGS per sample

def load_pgsc_calc_long_format(results_dir: Path, sample: str, pgs_ids: tuple, build: str) -> pd.DataFrame:
    """Load pgsc_calc results from long-format output files."""
    
    # Look for raw_scores.txt.gz (unadjusted scores)
    raw_file = results_dir / 'raw_scores.txt.gz'
    
    # Look for trent_pgs.txt.gz or {sample}_pgs.txt.gz (ancestry-normalized)
    norm_file = results_dir / f'{sample.lower()}_pgs.txt.gz'
    
    results = []
    
    # Load raw scores
    raw_df = None
    if raw_file.exists():
        raw_df = pd.read_csv(raw_file, sep='\t', compression='gzip')
        print(f"  Loaded raw scores: {len(raw_df)} rows")
    else:
        print(f"  Warning: raw_scores.txt.gz not found in {results_dir}")
    
    # Load ancestry-normalized scores
    norm_df = None
    if norm_file.exists():
        norm_df = pd.read_csv(norm_file, sep='\t', compression='gzip')
        # Filter to sample's rows (exclude 'reference' sampleset)
        norm_df = norm_df[norm_df['sampleset'] == sample.lower()].copy()
        print(f"  Loaded normalized scores: {len(norm_df)} rows for {sample}")
    else:
        print(f"  Warning: {sample.lower()}_pgs.txt.gz not found in {results_dir}")
    
    # Combine into results
    for pgs_id in pgs_ids:
        pgs_pattern = f"{pgs_id}_hmPOS_{build}"
        
        raw_score = None
        z_score = None
        percentile = None
        
        if raw_df is not None:
            pgs_row = raw_df[raw_df['PGS'].str.contains(pgs_id, na=False)]
            if not pgs_row.empty:
                raw_score = pgs_row.iloc[0]['SUM']
        
        if norm_df is not None:
            pgs_row = norm_df[norm_df['PGS'].str.contains(pgs_id, na=False)]
            if not pgs_row.empty:
                z_score = pgs_row.iloc[0].get('Z_MostSimilarPop', None)
                percentile = pgs_row.iloc[0].get('percentile_MostSimilarPop', None)
        
        if raw_score is not None or z_score is not None:
            results.append({
                'sample': sample,
                'pgs_id': pgs_id,
                'method': 'pgsc_calc',
                'build': build,
                'raw_score': raw_score,
                'z_score': z_score,
                'percentile': percentile
            })
    
    return pd.DataFrame(results)

# Load pgsc_calc results for Trent (Rowen doesn't have pgsc_calc results)
print("Loading pgsc_calc results (long format)...")
print()

pgsc_results = []

# Trent - has pgsc_calc ancestry results
trent_pgsc_dir = config.results_dir / 'pgsc_calc' / 'trent'
if trent_pgsc_dir.exists():
    print("Trent:")
    trent_pgsc = load_pgsc_calc_long_format(trent_pgsc_dir, 'Trent', config.pgs_ids, 'GRCh38')
    if not trent_pgsc.empty:
        pgsc_results.append(trent_pgsc)

# Combine
if pgsc_results:
    pgsc_df = pd.concat(pgsc_results, ignore_index=True)
    print(f"\nLoaded {len(pgsc_df)} pgsc_calc results:")
    display(pgsc_df)
else:
    pgsc_df = pd.DataFrame()
    print("No pgsc_calc results loaded.")

Loading pgsc_calc results (long format)...

Trent:
  Loaded raw scores: 4 rows
  Loaded normalized scores: 6 rows for Trent

Loaded 4 pgsc_calc results:


,sample,pgs_id,method,build,raw_score,z_score,percentile
0,Trent,PGS002308,pgsc_calc,GRCh38,0.436293,2.761803,99.700150
1,Trent,PGS004034,pgsc_calc,GRCh38,1.677837,0.490730,64.017991
2,Trent,PGS000027,pgsc_calc,GRCh38,17.716960,0.058853,51.574213
3,Trent,PGS004237,pgsc_calc,GRCh38,-0.258624,-1.468964,7.196402


### 3.5 Comprehensive Comparison: Custom vs pgsc_calc

Merge custom script results with pgsc_calc results to create a complete comparison table. This validates:
1. **Score equivalence**: pgsc_calc raw scores ≈ custom unadjusted scores (both ignore hom-ref sites)
2. **Bias magnitude**: How much the adjusted score differs from unadjusted
3. **Ancestry percentiles**: Where the sample falls in the European population distribution

In [12]:
# Create comprehensive comparison table
# Merge custom script results with pgsc_calc results

# Prepare custom results for merge
custom_for_merge = custom_results[['sample', 'pgs_id', 'trait', 'adjusted_score', 'unadjusted_score', 
                                    'homref_bias_pct', 'variants_from_vcf', 'variants_from_ref']].copy()

# Prepare pgsc_calc results for merge
if not pgsc_df.empty:
    pgsc_for_merge = pgsc_df[['sample', 'pgs_id', 'raw_score', 'z_score', 'percentile']].copy()
    pgsc_for_merge.columns = ['sample', 'pgs_id', 'pgsc_raw_score', 'pgsc_z_score', 'pgsc_percentile']
    
    # Merge
    comparison_df = custom_for_merge.merge(pgsc_for_merge, on=['sample', 'pgs_id'], how='left')
    
    # Calculate validation metrics
    comparison_df['raw_score_diff'] = comparison_df['pgsc_raw_score'] - comparison_df['unadjusted_score']
    comparison_df['raw_score_diff_pct'] = (comparison_df['raw_score_diff'] / comparison_df['unadjusted_score'].abs() * 100)
else:
    comparison_df = custom_for_merge.copy()
    comparison_df['pgsc_raw_score'] = None
    comparison_df['pgsc_z_score'] = None
    comparison_df['pgsc_percentile'] = None
    comparison_df['raw_score_diff'] = None
    comparison_df['raw_score_diff_pct'] = None

# Add interpretation column
def interpret_bias(bias_pct):
    if pd.isna(bias_pct):
        return "N/A"
    if bias_pct > 50:
        return "⚠️ SEVERE UNDERESTIMATE"
    elif bias_pct > 10:
        return "⚠️ Underestimate"
    elif bias_pct < -50:
        return "⚠️ SEVERE OVERESTIMATE"
    elif bias_pct < -10:
        return "⚠️ Overestimate"
    else:
        return "✓ Minimal bias"

comparison_df['interpretation'] = comparison_df['homref_bias_pct'].apply(interpret_bias)

print("=" * 100)
print("COMPREHENSIVE METHODOLOGY COMPARISON")
print("=" * 100)
print()
print("Key columns:")
print("  - adjusted_score: Custom script score WITH reference genome lookup (correct for WGS)")
print("  - unadjusted_score: Custom script score WITHOUT reference lookup (equivalent to pgsc_calc)")
print("  - pgsc_raw_score: pgsc_calc raw score (should match unadjusted_score)")
print("  - homref_bias_pct: Percentage difference showing systematic bias in pgsc_calc")
print()

display(comparison_df.round(3))

COMPREHENSIVE METHODOLOGY COMPARISON

Key columns:
  - adjusted_score: Custom script score WITH reference genome lookup (correct for WGS)
  - unadjusted_score: Custom script score WITHOUT reference lookup (equivalent to pgsc_calc)
  - pgsc_raw_score: pgsc_calc raw score (should match unadjusted_score)
  - homref_bias_pct: Percentage difference showing systematic bias in pgsc_calc



,sample,pgs_id,trait,adjusted_score,unadjusted_score,homref_bias_pct,variants_from_vcf,variants_from_ref,pgsc_raw_score,pgsc_z_score,pgsc_percentile,raw_score_diff,raw_score_diff_pct,interpretation
0,Trent,PGS002308,Type 2 Diabetes,0.392,0.437,-10.267,609023,650720,0.436,2.762,99.700,-0.001,-0.194,⚠️ Overestimate
1,Trent,PGS004034,Alzheimer's Disease,1.623,1.678,-3.297,556005,490903,1.678,0.491,64.018,-0.000,-0.000,✓ Minimal bias
2,Trent,PGS000027,BMI,38.640,17.718,118.088,1083426,1016742,17.717,0.059,51.574,-0.001,-0.004,⚠️ SEVERE UNDERESTIMATE
3,Trent,PGS004237,CAD,-0.270,-0.260,-4.082,601193,545306,-0.259,-1.469,7.196,0.001,0.470,✓ Minimal bias
4,Trent,PGS004696,CHD,-0.988,-0.185,-435.297,612790,677178,NaN,NaN,NaN,NaN,NaN,⚠️ SEVERE OVERESTIMATE


In [13]:
# Export comprehensive results to CSV
output_csv = config.results_dir / 'comprehensive_bias_analysis.csv'
comparison_df.to_csv(output_csv, index=False)
print(f"✅ Exported comparison data to: {output_csv}")

# Also create a summary table for the technical report
report_table = comparison_df[['sample', 'pgs_id', 'trait', 'adjusted_score', 'unadjusted_score', 
                               'homref_bias_pct', 'interpretation']].copy()
report_table.columns = ['Sample', 'PGS ID', 'Trait', 'Adjusted', 'Unadjusted', 'Bias %', 'Interpretation']

print("\n" + "=" * 80)
print("SUMMARY TABLE FOR TECHNICAL REPORT")
print("=" * 80)
display(report_table.round(2))

# Export summary table 
summary_csv = config.results_dir / 'bias_summary_for_report.csv'
report_table.to_csv(summary_csv, index=False)
print(f"\n✅ Exported summary table to: {summary_csv}")

✅ Exported comparison data to: results/methodology_comparison/comprehensive_bias_analysis.csv

SUMMARY TABLE FOR TECHNICAL REPORT


,Sample,PGS ID,Trait,Adjusted,Unadjusted,Bias %,Interpretation
0,Trent,PGS002308,Type 2 Diabetes,0.39,0.44,-10.27,⚠️ Overestimate
1,Trent,PGS004034,Alzheimer's Disease,1.62,1.68,-3.30,✓ Minimal bias
2,Trent,PGS000027,BMI,38.64,17.72,118.09,⚠️ SEVERE UNDERESTIMATE
3,Trent,PGS004237,CAD,-0.27,-0.26,-4.08,✓ Minimal bias
4,Trent,PGS004696,CHD,-0.99,-0.18,-435.30,⚠️ SEVERE OVERESTIMATE



✅ Exported summary table to: results/methodology_comparison/bias_summary_for_report.csv


In [14]:
# Key findings summary
print("=" * 80)
print("KEY FINDINGS: HOMOZYGOUS REFERENCE BIAS IN pgsc_calc")
print("=" * 80)

# Calculate aggregate statistics
avg_bias_abs = comparison_df['homref_bias_pct'].abs().mean()
max_underestimate = comparison_df[comparison_df['homref_bias_pct'] > 0]['homref_bias_pct'].max()
max_overestimate = comparison_df[comparison_df['homref_bias_pct'] < 0]['homref_bias_pct'].min()

# Most affected scores
most_biased = comparison_df.loc[comparison_df['homref_bias_pct'].abs().idxmax()]
bmi_bias = comparison_df[comparison_df['pgs_id'] == 'PGS000027']['homref_bias_pct'].mean()
chd_bias = comparison_df[comparison_df['pgs_id'] == 'PGS004696']['homref_bias_pct'].mean()

print(f"""
📊 AGGREGATE STATISTICS
────────────────────────────────────────────────────────────────
• Average absolute bias across all scores: {avg_bias_abs:.1f}%
• Maximum underestimation (positive bias): +{max_underestimate:.1f}%
• Maximum overestimation (negative bias): {max_overestimate:.1f}%

🔍 MOST AFFECTED SCORES
────────────────────────────────────────────────────────────────
• PGS000027 (BMI): {bmi_bias:+.1f}% bias - pgsc_calc SEVERELY UNDERESTIMATES
  → Effect alleles frequently match reference genome
  → Missing ~1M homozygous reference sites per sample
  
• PGS004696 (CHD): {chd_bias:+.1f}% bias - pgsc_calc SEVERELY OVERESTIMATES  
  → Effect alleles frequently differ from reference genome
  → Reference lookup reveals protective effect was missed

⚠️ CRITICAL INSIGHT
────────────────────────────────────────────────────────────────
The direction and magnitude of bias depends on how the PGS was constructed:
• Scores where effect alleles tend to be the non-reference allele → UNDERESTIMATE risk
• Scores where effect alleles tend to be the reference allele → OVERESTIMATE risk

Ancestry normalization does NOT correct this bias because the reference panel
(HGDP+1kGP) was scored using the same biased method!

✅ RECOMMENDATION
────────────────────────────────────────────────────────────────
For WGS data: Use reference genome lookup (custom script) for accurate PRS.
The "adjusted_score" column contains the corrected values.
""")

# Validation check: pgsc_calc raw should match custom unadjusted
if 'pgsc_raw_score' in comparison_df.columns and comparison_df['pgsc_raw_score'].notna().any():
    valid_rows = comparison_df[comparison_df['pgsc_raw_score'].notna()]
    score_diffs = (valid_rows['pgsc_raw_score'] - valid_rows['unadjusted_score']).abs()
    max_diff = score_diffs.max()
    print(f"\n✓ VALIDATION: pgsc_calc raw scores match custom unadjusted (max diff: {max_diff:.6f})")

KEY FINDINGS: HOMOZYGOUS REFERENCE BIAS IN pgsc_calc

📊 AGGREGATE STATISTICS
────────────────────────────────────────────────────────────────
• Average absolute bias across all scores: 114.2%
• Maximum underestimation (positive bias): +118.1%
• Maximum overestimation (negative bias): -435.3%

🔍 MOST AFFECTED SCORES
────────────────────────────────────────────────────────────────
• PGS000027 (BMI): +118.1% bias - pgsc_calc SEVERELY UNDERESTIMATES
  → Effect alleles frequently match reference genome
  → Missing ~1M homozygous reference sites per sample

• PGS004696 (CHD): -435.3% bias - pgsc_calc SEVERELY OVERESTIMATES  
  → Effect alleles frequently differ from reference genome
  → Reference lookup reveals protective effect was missed

⚠️ CRITICAL INSIGHT
────────────────────────────────────────────────────────────────
The direction and magnitude of bias depends on how the PGS was constructed:
• Scores where effect alleles tend to be the non-reference allele → UNDERESTIMATE risk
• Score

## Phase 4: Visualization

### 4.1 Score Comparison Plots

In [15]:
def plot_method_comparison(comparison_df: pd.DataFrame, output_path: Optional[Path] = None):
    """Create visualization comparing methods."""
    if comparison_df.empty:
        print("No comparison data available.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Plot 1: Custom Adjusted vs Unadjusted
    ax1 = axes[0, 0]
    for sample in comparison_df['sample'].unique():
        sample_df = comparison_df[comparison_df['sample'] == sample]
        ax1.scatter(sample_df['custom_unadjusted'], sample_df['custom_adjusted'], 
                   label=sample, s=100, alpha=0.7)
    
    # Add identity line
    lims = [min(ax1.get_xlim()[0], ax1.get_ylim()[0]), 
            max(ax1.get_xlim()[1], ax1.get_ylim()[1])]
    ax1.plot(lims, lims, 'k--', alpha=0.5, label='y=x')
    ax1.set_xlabel('Custom Unadjusted (VCF only)')
    ax1.set_ylabel('Custom Adjusted (+ ref lookup)')
    ax1.set_title('Custom Script: Impact of Reference Lookup')
    ax1.legend()
    
    # Plot 2: pgsc_calc Raw vs Custom Unadjusted
    ax2 = axes[0, 1]
    for sample in comparison_df['sample'].unique():
        sample_df = comparison_df[comparison_df['sample'] == sample]
        ax2.scatter(sample_df['custom_unadjusted'], sample_df['pgsc_calc_raw'],
                   label=sample, s=100, alpha=0.7)
    
    lims = [min(ax2.get_xlim()[0], ax2.get_ylim()[0]),
            max(ax2.get_xlim()[1], ax2.get_ylim()[1])]
    ax2.plot(lims, lims, 'k--', alpha=0.5, label='y=x')
    ax2.set_xlabel('Custom Unadjusted')
    ax2.set_ylabel('pgsc_calc Raw')
    ax2.set_title('Method Comparison: Should Be ~Equal')
    ax2.legend()
    
    # Plot 3: Homozygous Reference Bias by PGS
    ax3 = axes[1, 0]
    bias_pivot = comparison_df.pivot(index='pgs_id', columns='sample', values='homref_bias_pct')
    bias_pivot.plot(kind='bar', ax=ax3)
    ax3.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax3.set_ylabel('Bias (%)')
    ax3.set_title('Homozygous Reference Bias by Score')
    ax3.tick_params(axis='x', rotation=45)
    
    # Plot 4: Coverage vs Bias
    ax4 = axes[1, 1]
    scatter = ax4.scatter(comparison_df['variants_from_ref'], 
                         comparison_df['homref_bias_pct'],
                         c=comparison_df['sample'].astype('category').cat.codes,
                         s=100, alpha=0.7)
    ax4.set_xlabel('Variants from Reference Lookup')
    ax4.set_ylabel('Homozygous Reference Bias (%)')
    ax4.set_title('More Ref Lookups → More Potential Bias')
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        print(f"Saved figure to {output_path}")
    
    plt.show()

# Run when results are available
# plot_method_comparison(bias_df, config.results_dir / 'method_comparison.png')

### 4.2 Ancestry Correction Deep Dive

Key question: Does pgsc_calc's ancestry normalization account for the homozygous reference bias?

In [16]:
def analyze_ancestry_correction(comparison_df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyze whether ancestry correction accounts for homref bias.
    
    Theory:
    - If the reference panel (HGDP+1kGP) was scored WITH reference lookup,
      then pgsc_calc's normalization should account for the bias.
    - If scored WITHOUT reference lookup (likely), the bias persists.
    
    Test: Compare Z-score implied by custom adjusted score vs pgsc_calc Z-score.
    """
    if comparison_df.empty or 'pgsc_calc_z' not in comparison_df.columns:
        return pd.DataFrame()
    
    analysis_data = []
    
    for _, row in comparison_df.iterrows():
        if pd.isna(row['pgsc_calc_z']):
            continue
        
        # pgsc_calc Z-score is based on raw (unadjusted-equivalent) score
        pgsc_z = row['pgsc_calc_z']
        
        # If we applied the same normalization to the adjusted score,
        # we'd expect a different Z-score
        # Approximation: Z_adjusted ≈ Z_raw + (bias_pct / 100) * some_factor
        # This is a simplification - actual relationship depends on score distribution
        
        homref_bias = row['homref_bias_pct'] if row['homref_bias_pct'] else 0
        
        analysis_data.append({
            'sample': row['sample'],
            'pgs_id': row['pgs_id'],
            'pgsc_calc_z': pgsc_z,
            'pgsc_calc_percentile': row['pgsc_calc_percentile'],
            'homref_bias_pct': homref_bias,
            'interpretation': _interpret_bias(homref_bias, pgsc_z)
        })
    
    return pd.DataFrame(analysis_data)

def _interpret_bias(bias_pct: float, z_score: float) -> str:
    """Provide interpretation of bias significance."""
    if abs(bias_pct) < 1:
        return "Minimal bias - results reliable"
    elif abs(bias_pct) < 5:
        return "Low bias - minor adjustment may be needed"
    elif abs(bias_pct) < 20:
        return "Moderate bias - interpret with caution"
    else:
        return "HIGH BIAS - pgsc_calc results unreliable for this score"

# Run when results are available
# ancestry_analysis = analyze_ancestry_correction(bias_df)
# display(ancestry_analysis)

## Phase 5: Summary and Conclusions

In [17]:
def generate_summary_report(comparison_df: pd.DataFrame, output_path: Optional[Path] = None) -> str:
    """Generate a markdown summary report."""
    if comparison_df.empty:
        return "No results available for summary."
    
    lines = [
        "# PRS Methodology Experiment: Summary Report",
        "",
        "## Key Findings",
        "",
        "### Homozygous Reference Bias",
        "",
        "| Sample | PGS ID | Bias (%) | Interpretation |",
        "|--------|--------|----------|----------------|",
    ]
    
    for _, row in comparison_df.iterrows():
        bias = row.get('homref_bias_pct', 0)
        interp = _interpret_bias(bias, 0)
        lines.append(f"| {row['sample']} | {row['pgs_id']} | {bias:.1f}% | {interp} |")
    
    lines.extend([
        "",
        "### pgsc_calc vs Custom Script Agreement",
        "",
        "The pgsc_calc raw scores should closely match the custom script's 'unadjusted' scores,",
        "as both ignore homozygous reference sites not present in the VCF.",
        "",
        "### Ancestry Correction Behavior",
        "",
        "**Critical Question**: Does pgsc_calc's ancestry normalization correct for the homref bias?",
        "",
        "**Answer**: Most likely **NO**. The HGDP+1kGP reference panel was scored using standard",
        "PLINK scoring, which also ignores homozygous reference sites. Therefore:",
        "",
        "- The bias affects both the sample AND the reference panel equally",
        "- Z-scores and percentiles remain biased in the same direction",
        "- For scores where effect alleles are systematically REF-oriented, pgsc_calc will",
        "  **underestimate** genetic risk for WGS data",
        "",
        "## Recommendations",
        "",
        "1. **For WGS data**: Use the custom script's adjusted scores for accurate PRS values",
        "2. **For ancestry normalization**: Apply custom normalization using adjusted scores",
        "3. **For array data**: pgsc_calc is appropriate (designed for imputed data)",
        "4. **Score selection**: Prefer scores with balanced effect allele orientation",
    ])
    
    report = "\n".join(lines)
    
    if output_path:
        with open(output_path, 'w') as f:
            f.write(report)
        print(f"Report saved to {output_path}")
    
    return report

# Run when results are available
# report = generate_summary_report(bias_df, config.results_dir / 'summary_report.md')
# print(report)

## Appendix: Understanding pgsc_calc Ancestry Correction

Based on [Lambert et al. 2024](https://doi.org/10.1101/2024.05.29.24307783), pgsc_calc implements these normalization methods:

### Population-Based Normalization
1. Project sample into PCA space using HGDP+1kGP reference
2. Identify most similar population (EUR, AFR, EAS, etc.)
3. Calculate Z-score: `(sample_PGS - pop_mean) / pop_sd`
4. Convert to percentile

### Continuous (Regression-Based) Normalization
1. Fit regression: `PGS ~ PC1 + PC2 + ... + PCn` on reference panel
2. Predict expected PGS for sample based on PCA loadings
3. Calculate residual as ancestry-adjusted PGS

### Why Neither Method Fixes the Homref Bias

Both methods compare the sample's score to reference panel scores. If both were calculated
using the same method (VCF-only, no reference lookup), the comparison is **internally consistent**
but **externally biased**.

The Z-score tells you where you fall relative to others scored the same way - not your true
genetic predisposition.

### Implications for Clinical Use

For clinical risk stratification:
- Raw Z-scores from pgsc_calc may underestimate risk for WGS samples
- The bias magnitude depends on the score's effect allele orientation
- Scores using PRS-CS/PRS-CSx tend to have more balanced orientation
- Scores using p-value thresholding may have systematic REF bias